In the previous Notebooks i've established that using a Linear regression to predict Rank is my best bet.

This method is called  a pointwise ranking method. and the "learn to rank" Theory proposes more ranking methods that i'll try to explore in this norebook. 
    -Pairwise ranking
    -Listwise ranking

Let's start with Pairwise ranking there are various methods all from Microsoft researcher Burges
RankNet, LambdaRank, LambdaRankMart